In [311]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

In [312]:
# define and read may 2019 citibike data
file_one = "Downloads/201905-citibike-tripdata.csv"

In [313]:
df = pd.read_csv(file_one, encoding="ISO-8859-1")

In [314]:
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,139,2019-05-01 00:00:01.9010,2019-05-01 00:02:21.5170,447,8 Ave & W 52 St,40.763707,-73.985162,423,W 54 St & 9 Ave,40.765849,-73.986905,31170,Subscriber,1983,1
1,754,2019-05-01 00:00:03.0210,2019-05-01 00:12:37.6920,3258,W 27 St & 10 Ave,40.750182,-74.002184,3255,8 Ave & W 31 St,40.750585,-73.994685,25560,Customer,1969,0
2,2308,2019-05-01 00:00:04.6270,2019-05-01 00:38:33.1710,3093,N 6 St & Bedford Ave,40.717452,-73.958509,3676,Van Brunt St & Van Dyke St,40.675833,-74.014726,33369,Subscriber,1978,1
3,143,2019-05-01 00:00:19.3340,2019-05-01 00:02:42.5200,3486,Schermerhorn St & Bond St,40.688417,-73.984517,3412,Pacific St & Nevins St,40.685376,-73.983021,32041,Subscriber,1997,1
4,138,2019-05-01 00:00:22.1840,2019-05-01 00:02:40.6480,388,W 26 St & 10 Ave,40.749718,-74.002950,494,W 26 St & 8 Ave,40.747348,-73.997236,35237,Subscriber,1967,1


In [315]:
df.columns

Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender'],
      dtype='object')

In [316]:
df.dtypes

tripduration                 int64
starttime                   object
stoptime                    object
start station id             int64
start station name          object
start station latitude     float64
start station longitude    float64
end station id               int64
end station name            object
end station latitude       float64
end station longitude      float64
bikeid                       int64
usertype                    object
birth year                   int64
gender                       int64
dtype: object

In [317]:
#Rename some columns for easy manipulation
df = df.rename(index=str, columns={"start station id": "id", "start station latitude": "latitude", "start station longitude": "longitude", "end station id": "id2", "end station latitude": "lat2", "end station longitude": "lon2"})
df.head()


,tripduration,starttime,stoptime,id,start station name,latitude,longitude,id2,end station name,lat2,lon2,bikeid,usertype,birth year,gender
0,139,2019-05-01 00:00:01.9010,2019-05-01 00:02:21.5170,447,8 Ave & W 52 St,40.763707,-73.985162,423,W 54 St & 9 Ave,40.765849,-73.986905,31170,Subscriber,1983,1
1,754,2019-05-01 00:00:03.0210,2019-05-01 00:12:37.6920,3258,W 27 St & 10 Ave,40.750182,-74.002184,3255,8 Ave & W 31 St,40.750585,-73.994685,25560,Customer,1969,0
2,2308,2019-05-01 00:00:04.6270,2019-05-01 00:38:33.1710,3093,N 6 St & Bedford Ave,40.717452,-73.958509,3676,Van Brunt St & Van Dyke St,40.675833,-74.014726,33369,Subscriber,1978,1
3,143,2019-05-01 00:00:19.3340,2019-05-01 00:02:42.5200,3486,Schermerhorn St & Bond St,40.688417,-73.984517,3412,Pacific St & Nevins St,40.685376,-73.983021,32041,Subscriber,1997,1
4,138,2019-05-01 00:00:22.1840,2019-05-01 00:02:40.6480,388,W 26 St & 10 Ave,40.749718,-74.002950,494,W 26 St & 8 Ave,40.747348,-73.997236,35237,Subscriber,1967,1


In [318]:
#make a copy df with necessary columns only
#new is the start lats/lons
new = df[['id', 'latitude', 'longitude']].copy()
#new_end is the end lats/lons
new_end = df[['id2', 'lat2', 'lon2']].copy()
#change names back to be consistent below
new_end = new_end.rename(index=str, columns={"id2": "id", "lat2": "latitude", "lon2": "longitude"})
#drop duplicates from both dataframes
new = new.drop_duplicates()
new_end = new_end.drop_duplicates()
print(new.shape[0])
print(new.shape[1])
print(new_end.shape[0])
print(new_end.shape[1])

793
3
804
3


In [319]:
# Use a shape file with zip code boundaries in NYC
file_two = gpd.read_file("Downloads/ZIP_CODE_040114/ZIP_CODE_040114.shp")
print(file_two.columns)
#hold_zip_df holds just geometry and zipcode data
hold_zip_df = file_two[['ZIPCODE', 'geometry']].copy()

print(hold_zip_df.shape[0])
print(hold_zip_df.dtypes)

# make the zipcode a string
hold_zip_df['ZIPCODE'] = hold_zip_df['ZIPCODE'].astype(str)

print(hold_zip_df.dtypes)

# make another copy with correct crs info
zip_df = hold_zip_df.to_crs({'init': 'epsg:4326'}) 
zip_df.head()


Index(['ZIPCODE', 'BLDGZIP', 'PO_NAME', 'POPULATION', 'AREA', 'STATE',
       'COUNTY', 'ST_FIPS', 'CTY_FIPS', 'URL', 'SHAPE_AREA', 'SHAPE_LEN',
       'geometry'],
      dtype='object')
263
ZIPCODE     object
geometry    object
dtype: object
ZIPCODE     object
geometry    object
dtype: object


,ZIPCODE,geometry
0,11436,POLYGON ((-73.80584847647394 40.68290932644251...
1,11213,POLYGON ((-73.93739763139813 40.67972958925087...
2,11212,POLYGON ((-73.90294132545438 40.67083977590012...
3,11225,POLYGON ((-73.95797316043482 40.67065695897571...
4,11218,POLYGON ((-73.97208109564257 40.65059658727614...


In [320]:
# use geopandas to get a geoDataFrame 
crs = {'init' :'epsg:4326'}
gdf = gpd.GeoDataFrame(new, crs=crs, geometry=gpd.points_from_xy(new.longitude, new.latitude))
print(gdf.dtypes)
gdf.head()

id             int64
latitude     float64
longitude    float64
geometry      object
dtype: object


,id,latitude,longitude,geometry
0,447,40.763707,-73.985162,POINT (-73.9851615 40.76370739)
1,3258,40.750182,-74.002184,POINT (-74.00218427181244 40.75018156325682)
2,3093,40.717452,-73.958509,POINT (-73.95850939 40.71745169)
3,3486,40.688417,-73.984517,POINT (-73.98451656103134 40.68841742754083)
4,388,40.749718,-74.002950,POINT (-74.00295034600001 40.749717753)


In [321]:
gdf.loc[gdf['id'] == 3526]

,id,latitude,longitude,geometry
413,3526,40.774788,-73.912555,POINT (-73.91255509999999 40.7747878)


In [322]:
gdf_start = gpd.sjoin(gdf, zip_df, how='left', op="within")

In [323]:
gdf_start.head()

,id,latitude,longitude,geometry,index_right,ZIPCODE
0,447,40.763707,-73.985162,POINT (-73.9851615 40.76370739),187.0,10019
1,3258,40.750182,-74.002184,POINT (-74.00218427181244 40.75018156325682),82.0,10001
2,3093,40.717452,-73.958509,POINT (-73.95850939 40.71745169),260.0,11249
3,3486,40.688417,-73.984517,POINT (-73.98451656103134 40.68841742754083),108.0,11201
4,388,40.749718,-74.002950,POINT (-74.00295034600001 40.749717753),82.0,10001


In [324]:
print(gdf.crs)
print(zip_df.crs)

{'init': 'epsg:4326'}
{'init': 'epsg:4326'}


In [228]:
# write a new file with joined zipcodes
#citibike_with_zip_csv = gdf_start.to_csv (r'/Users/amyclaman/Desktop/citibike_with_zip2.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path



In [326]:
zdf = gdf_start[['id', 'ZIPCODE']].copy()

In [270]:
#zip_file = "/Users/amyclaman/Downloads/citibike_with_zip.csv"
#zip_df = pd.read_csv(zip_file, encoding="ISO-8859-1")

In [271]:
#zdf = zip_df[['id', 'ZIPCODE']].copy()

In [327]:
zdf.dtypes

id          int64
ZIPCODE    object
dtype: object

In [329]:
# clean zip code data..define as integers, manipulate nulls
zdf.ZIPCODE = zdf.ZIPCODE.fillna(-1)
zdf.ZIPCODE = zdf.ZIPCODE.astype(int)
zdf.ZIPCODE = zdf.ZIPCODE.astype(str)
zdf.ZIPCODE = zdf.ZIPCODE.replace('-1', np.nan)


In [330]:
zdf.head()

,id,ZIPCODE
0,447,10019
1,3258,10001
2,3093,11249
3,3486,11201
4,388,10001


In [382]:
# find zipcodes 83....not a 'real' nyc zip....find and fix
zdf.ZIPCODE[zdf.id==3143] = '10021'
zdf.loc[zdf.ZIPCODE == '83']

/Users/amyclaman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,id,ZIPCODE


In [331]:
# put column headers back to original values
df = df.rename(index=str, columns={"id": "start station id", "latitude": "start station latitude", "longitude": "start station longitude", "id2": "end station id", "lat2": "end station latitude","lon2": "end station longitude"})
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,139,2019-05-01 00:00:01.9010,2019-05-01 00:02:21.5170,447,8 Ave & W 52 St,40.763707,-73.985162,423,W 54 St & 9 Ave,40.765849,-73.986905,31170,Subscriber,1983,1
1,754,2019-05-01 00:00:03.0210,2019-05-01 00:12:37.6920,3258,W 27 St & 10 Ave,40.750182,-74.002184,3255,8 Ave & W 31 St,40.750585,-73.994685,25560,Customer,1969,0
2,2308,2019-05-01 00:00:04.6270,2019-05-01 00:38:33.1710,3093,N 6 St & Bedford Ave,40.717452,-73.958509,3676,Van Brunt St & Van Dyke St,40.675833,-74.014726,33369,Subscriber,1978,1
3,143,2019-05-01 00:00:19.3340,2019-05-01 00:02:42.5200,3486,Schermerhorn St & Bond St,40.688417,-73.984517,3412,Pacific St & Nevins St,40.685376,-73.983021,32041,Subscriber,1997,1
4,138,2019-05-01 00:00:22.1840,2019-05-01 00:02:40.6480,388,W 26 St & 10 Ave,40.749718,-74.002950,494,W 26 St & 8 Ave,40.747348,-73.997236,35237,Subscriber,1967,1


In [280]:

#df["start station id"] = df["start station id"].fillna(-1)
#df["start station id"] = df["start station id"].astype(int)
#df["start station id"] = df["start station id"].astype(str)
#df["start station id"] = df["start station id"].replace('-1', np.nan)

In [281]:

#df["end station id"] = df["end station id"].fillna(-1)
#df["end station id"] = df["end station id"].astype(int)
#df["end station id"] = df["end station id"].astype(str)
#df["end station id"] = df["end station id"].replace('-1', np.nan)

In [383]:
# create column with start zipcodes in orig file
df['start zip'] = df['start station id'].map(zdf.set_index('id')['ZIPCODE'])
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,start zip,end zip
0,139,2019-05-01 00:00:01.9010,2019-05-01 00:02:21.5170,447,8 Ave & W 52 St,40.763707,-73.985162,423,W 54 St & 9 Ave,40.765849,-73.986905,31170,Subscriber,1983,1,10019,10019
1,754,2019-05-01 00:00:03.0210,2019-05-01 00:12:37.6920,3258,W 27 St & 10 Ave,40.750182,-74.002184,3255,8 Ave & W 31 St,40.750585,-73.994685,25560,Customer,1969,0,10001,10001
2,2308,2019-05-01 00:00:04.6270,2019-05-01 00:38:33.1710,3093,N 6 St & Bedford Ave,40.717452,-73.958509,3676,Van Brunt St & Van Dyke St,40.675833,-74.014726,33369,Subscriber,1978,1,11249,11231
3,143,2019-05-01 00:00:19.3340,2019-05-01 00:02:42.5200,3486,Schermerhorn St & Bond St,40.688417,-73.984517,3412,Pacific St & Nevins St,40.685376,-73.983021,32041,Subscriber,1997,1,11201,11217
4,138,2019-05-01 00:00:22.1840,2019-05-01 00:02:40.6480,388,W 26 St & 10 Ave,40.749718,-74.002950,494,W 26 St & 8 Ave,40.747348,-73.997236,35237,Subscriber,1967,1,10001,10001


In [384]:
# create a column with end station zip codes in orig file
df['end zip'] = df['end station id'].map(zdf.set_index('id')['ZIPCODE'])
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,start zip,end zip
0,139,2019-05-01 00:00:01.9010,2019-05-01 00:02:21.5170,447,8 Ave & W 52 St,40.763707,-73.985162,423,W 54 St & 9 Ave,40.765849,-73.986905,31170,Subscriber,1983,1,10019,10019
1,754,2019-05-01 00:00:03.0210,2019-05-01 00:12:37.6920,3258,W 27 St & 10 Ave,40.750182,-74.002184,3255,8 Ave & W 31 St,40.750585,-73.994685,25560,Customer,1969,0,10001,10001
2,2308,2019-05-01 00:00:04.6270,2019-05-01 00:38:33.1710,3093,N 6 St & Bedford Ave,40.717452,-73.958509,3676,Van Brunt St & Van Dyke St,40.675833,-74.014726,33369,Subscriber,1978,1,11249,11231
3,143,2019-05-01 00:00:19.3340,2019-05-01 00:02:42.5200,3486,Schermerhorn St & Bond St,40.688417,-73.984517,3412,Pacific St & Nevins St,40.685376,-73.983021,32041,Subscriber,1997,1,11201,11217
4,138,2019-05-01 00:00:22.1840,2019-05-01 00:02:40.6480,388,W 26 St & 10 Ave,40.749718,-74.002950,494,W 26 St & 8 Ave,40.747348,-73.997236,35237,Subscriber,1967,1,10001,10001


In [403]:
#find nulls
df.isnull().sum()

tripduration               0
starttime                  0
stoptime                   0
start station id           0
start station name         0
start station latitude     0
start station longitude    0
end station id             0
end station name           0
end station latitude       0
end station longitude      0
bikeid                     0
usertype                   0
birth year                 0
gender                     0
start zip                  0
end zip                    3
dtype: int64

In [406]:
df[df.isnull().any(axis=1)]

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,start zip,end zip
1396333,338,2019-05-23 15:56:16.7700,2019-05-23 16:01:55.4000,514,12 Ave & W 40 St,40.760875,-74.002777,3240,NYCBS Depot BAL - DYR,40.75903,-73.993859,33290,Subscriber,1990,1,10019,NaN


In [405]:
# find and manually replace missing zip codes
df.loc[df['end station id'] == 315, 'end zip'] = "10005"
df.loc[df['end station id'] == 3245, 'end zip'] = "10002"
df.loc[df['end station id'] == 3205, 'end zip'] = "07302"
df.loc[df['end station id'] == 3275, 'end zip'] = "07302"
df.loc[df['end station id'] == 3681, 'end zip'] = "07302"
df.loc[df['end station id'] == 3183, 'end zip'] = "07302"
df.loc[df['end station id'] == 3184, 'end zip'] = "07302"
df.loc[df['end station id'] == 3426, 'end zip'] = "07304"
df.loc[df['end station id'] == 3199, 'end zip'] = "07310"
df.loc[df['end station id'] == 3198, 'end zip'] = "07307"
df.loc[df['end station id'] == 3779, 'end zip'] = "11237"

In [407]:
# write file
citibike_zip_csv = df.to_csv (r'/Users/amyclaman/Desktop/citibike_zipcode_201905.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path


In [387]:
# double check for bogus zipcode
df.loc[df['start zip'] == 83]

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,start zip,end zip
